In [10]:
#HIDE
try:
    import setup
except:
    pass
from notebook_helper import *
notebook_setup()

from modules import la
from ipywidgets import interactive
from IPython.display import clear_output

# Council emissions profile demo

This notebook is a quick demo of v1 of the emissions profile and distance tool. This may take a few seconds to load.

In [11]:
dmd = lambda x: display(md(x))

dmd("Loading...")

# prepare decile guide
cdf = pd.read_csv(Path("data",  "old_council_clusters", "council_clusters_v1.csv"))
cdf = cdf.la.get_council_info(["pop-2020"]).set_index(
    "local-authority-code")

for c in cdf.columns:
    if "Total" in c:
        name = " ".join(c.split(" ")[:-1]) +" Emissions Decile"
        per_capita = cdf[c] / cdf["pop-2020"]
        cdf[name] = pd.qcut(per_capita, 10,
                                    labels=range(1, 11), duplicates="drop")

ddf = pd.read_csv(Path("data", "old_council_clusters", "council_clusters_v1_distance.csv")) #"council_clusters_v1_distance.csv"


def standardise_distance(df):
    df = df.copy()
    # use tenth from last because the last point might be an extreme outlier (in this case london)
    tenth_from_last_score = df["distance"].sort_values().tail(10).iloc[0]
    df["match"] = 1 - (df["distance"] / tenth_from_last_score)
    df["match"] = df["match"].round(3) * 100
    df = df.sort_values("match", ascending=False)
    df = df.drop(columns="local-authority-code_A")
    return df

ddf = ddf.groupby("local-authority-code_A").apply(standardise_distance)

def display_council(council_slug):
    wanted_columns = ["official-name", "match",
                    "local-authority-type-name", "label"]

    matches = ddf.loc[council_slug].sort_values("match", ascending=False).head(10)
    matches = matches.set_index("local-authority-code_B").join(cdf).reset_index()
    matches = matches[wanted_columns]

    details = cdf.loc[council_slug].to_dict()

    dmd(f'## {details["official-name"]}')
    dmd(f'**Type**: {details["local-authority-type-name"]}')
    dmd(f'**Cluster**: {details["label"]}')
    dmd(f'**Cluster description**: {details["label_desc"]}')

    deciles = cdf.loc[cdf.index == council_slug][[x for x in cdf.columns if "Decile" in x]]
    deciles = deciles.transpose().rename(columns={council_slug:"Decile"})
    dmd("### Decile profiles")
    dmd("Decile 10 is the highest emissions, decile 1 is the lowest.")

    display(deciles)

    dmd("### Nearest neighbours")
    display(matches)

lookup = cdf.reset_index().set_index("official-name")["local-authority-code"].to_dict()
clear_output()
interactive(lambda council: display_council(lookup[council]), council=list(cdf["official-name"].sort_values()))

interactive(children=(Dropdown(description='council', options=('Aberdeen City Council', 'Aberdeenshire Council…